In [92]:
import pysqlite3
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
import time
import jsondiff
import argparse
import pandas as pd
import os

from langchain_chroma import Chroma
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings
from ollama import Client
#from openai import OpenAI
from secret import OPENAI_API_KEY
import json, jsondiff
import numpy as np
from sklearn.model_selection import train_test_split

In [93]:
NEW_PROMPT1 = """Your task is to transform natural language network intents into JSON-formatted network policies compatible with the OpenDaylight (ODL) SDN controller's configuration datastore.

You only reply in JSON, no natural language. The network intents can represent different traffic control behaviors, such as:

a. **Traffic Forwarding:** Specify how traffic with particular characteristics (e.g., IPv4 destination, TCP port) should be forwarded through specific ports.  
b. **Firewall Blocking:** Define rules to drop traffic based on specific match criteria (e.g., source IP, destination IP).  
c. **Port-Based Forwarding:** Redirect traffic entering a specific port to another designated port.

### JSON STRUCTURE:

1. **Traffic Forwarding Rule:**  

```json
{
  "flow-node-inventory:flow": [
    {
      "id": "<unique_id>",
      "priority": <integer>,
      "table_id": <integer>,
      "flow-name": "<descriptive_name>",
      "hard-timeout": <integer>,
      "idle-timeout": <integer>,
      "match": {
        "ethernet-match": {
          "ethernet-type": {
            "type": <integer>
          }
        },
        "ipv4-destination": "<ip_address/mask>",
        "ip-match": {
          "ip-protocol": <integer>
        },
        "tcp-destination-port": <integer>
      },
      "instructions": {
        "instruction": [
          {
            "order": 0,
            "apply-actions": {
              "action": [
                {
                  "order": 0,
                  "output-action": {
                    "output-node-connector": "<port_number>"
                  }
                }
              ]
            }
          }
        ]
      }
    }
  ]
}

2. **Blocking or Dropping Rule:**  

{
  "flow-node-inventory:flow": [
    {
      "id": "<unique_id>",
      "priority": <integer>,
      "table_id": <integer>,
      "flow-name": "<descriptive_name>",
      "hard-timeout": <integer>,
      "idle-timeout": <integer>,
      "match": {
        "ethernet-match": {
          "ethernet-type": {
            "type": <integer>
          }
        },
        "ipv4-source": "<ip_address/mask>",
        "ipv4-destination": "<ip_address/mask>"
      },
      "instructions": {
        "instruction": [
          {
            "order": 0,
            "apply-actions": {
              "action": [
                {
                  "order": 0,
                  "drop-action": {}
                }
              ]
            }
          }
        ]
      }
    }
  ]
}

3. **Port-Based Forwarding Rule:**  

{
  "flow-node-inventory:flow": [
    {
      "id": "<unique_id>",
      "priority": <integer>,
      "table_id": <integer>,
      "flow-name": "<descriptive_name>",
      "hard-timeout": <integer>,
      "idle-timeout": <integer>,
      "match": {
        "in-port": <port_number>
      },
      "instructions": {
        "instruction": [
          {
            "order": 0,
            "apply-actions": {
              "action": [
                {
                  "order": 0,
                  "output-action": {
                    "output-node-connector": "<port_number>"
                  }
                }
              ]
            }
          }
        ]
      }
    }
  ]
}

Field Descriptions: 
id: A number representing a unique identifier for the flow (0 for default).
priority: Priority level (higher numbers indicate higher priority).
table_id: An integer representing the flow table identifier (0 for default).
flow-name: A short, descriptive flow name.
hard-timeout: Timeout in seconds after which the flow is removed (0 for default).
idle-timeout: Timeout in seconds after which the flow is removed if there's no activity (0 for default).
ethernet-type: Ethernet protocol type (e.g., 2048 for IPv4).
ipv4-destination: IPv4 address in CIDR notation (e.g., 10.0.0.1/32 (optional)).
ipv4-source: Source IP address (optional).
ip-protocol: Use "ip-match" and "ip-protocol" when specifying specific transport layer protocols. (e.g., 6 for TCP, 17 for UDP, 1 for ICMP (optional)).
tcp-source-port: TCP port number (optional).
tcp-destination-port: TCP port number (optional).
in-port: A value representing incoming interface port number (optional).
output-action: Use "output-action" and "output-node-connector" to specify the output port number.
drop-action: Use {} to indicate packet dropping.

RULES: 
Each id must be unique.
Set priority values appropriately (higher for critical rules, lower for defaults).
Use valid match conditions (ipv4-destination, tcp-destination-port, ipv4-source, in-port) depending on the intent type.
Ensure valid ODL-compliant JSON syntax.
Avoid duplicate keys and empty fields.
Verify JSON structure for correctness before responding.
Always respond with valid JSON only, with no additional text, comments, or explanations.
If the intent cannot be mapped, return an empty JSON object {}."""

In [94]:
NEW_PROMPT2 = """Your task is to translate natural language descriptions of network traffic rules into JSON formatted flow entries suitable for an OpenDaylight (ODL) SDN controller.

You will receive natural language descriptions specifying how network traffic should be handled. Your output MUST be valid JSON representing one or more flow entries. Each flow entry defines a rule for processing network packets.

The JSON output should adhere to the following structure, which is compatible with ODL's configuration datastore:

```json
{
  "flow-node-inventory:flow": [ // Or just "flow" for single flow entries
    {
      "id": "<unique_id>", // A unique identifier for the flow (e.g., a number or string)
      "priority": <priority_number>, // An integer representing the flow's priority (higher number = higher priority)
      "table_id": 0, // Usually 0 for basic forwarding rules
      "flow-name": "<descriptive_name>", // A descriptive name for the flow
      "hard-timeout": 0, // Timeout in seconds after which the flow is removed (0 for no timeout)
      "idle-timeout": 0, // Timeout in seconds after which the flow is removed if there's no activity (0 for no timeout)
      "match": { // Criteria for matching packets
        "in-port": <port_number>, // The input port (optional)
        "ethernet-match": {
          "ethernet-type": {
            "type": <ethernet_type_code> // e.g., 2048 for IPv4, 34525 for IPv6
          }
        },
        "ipv4-source": "<ip_address>/<prefix_length>", // Source IPv4 address and prefix (optional)
        "ipv4-destination": "<ip_address>/<prefix_length>", // Destination IPv4 address and prefix (optional)
        "ip-match": {
          "ip-protocol": <ip_protocol_number> // e.g., 6 for TCP, 17 for UDP, 1 for ICMP (optional)
        },
        "tcp-source-port": <port_number>, // Source TCP port (optional)
        "tcp-destination-port": <port_number> // Destination TCP port (optional)
      },
      "instructions": { // Actions to be performed on matching packets
        "instruction": [
          {
            "order": 0,
            "apply-actions": {
              "action": [
                {
                  "order": 0,
                  "output-action": {
                    "output-node-connector": "<port_number>" // The output port
                  }
                },
                {
                  "order":0,
                  "drop-action": {} // Action to drop the packet
                }
              ]
            }
          }
        ]
      }
    }
  ]
}

Key points to remember:

Ethernet Type: Include "ethernet-match" and "ethernet-type" with the correct type code (e.g., 2048 for IPv4) whenever dealing with IP traffic.
IP Protocol: Use "ip-match" and "ip-protocol" (e.g., 6 for TCP, 1 for ICMP) when specifying transport layer protocols.
Output Action: Use "output-action" and "output-node-connector" to specify the output port.
Drop Action: Use "drop-action" to drop the packet.
Priority: Use "priority" to set the priority of the rule. Higher numbers mean higher priority.
Flow ID and Name: Provide a unique "id" and a descriptive "flow-name" for each flow.
Single vs. Multiple Flows: If the intent describes only one flow, you can use the structure {"flow": {...}}. If the intent implies multiple flows, use {"flow-node-inventory:flow": [{...}, {...}, ...]}.
No other text: Only output valid JSON. Do not add any explanations or other text."""

In [95]:
NEW_PROMPT3 = """Your task is to transform natural language network intents into JSON-formatted network policies compatible with the OpenDaylight (ODL) SDN controller's configuration datastore.

You only reply in JSON, no natural language. The network intents can represent different traffic control behaviors, such as:

a. **Traffic Forwarding and Queue-Based Traffic Control Rule:** Define rules for forwarding traffic based on IPv4 destination, TCP/UDP ports, and optionally assign traffic to specific queues for prioritization. 
b. **Firewall Blocking:** Define rules to drop traffic based on specific match criteria (e.g., source IP, destination IP).  
c. **Port-Based Forwarding:** Redirect traffic entering a specific port to another designated port.

### JSON STRUCTURE:

1. **Traffic Forwarding and Queue-Based Traffic Control Rule:**  

```json
{
  "flow-node-inventory:flow": [
    {
      "id": "<unique_id>",
      "priority": <integer>,
      "table_id": <integer>,
      "flow-name": "<descriptive_name>",
      "hard-timeout": <integer>,
      "idle-timeout": <integer>,
      "match": {
        "ethernet-match": {
          "ethernet-type": {
            "type": 2048
          }
        },
        "ipv4-destination": "<ip_address/mask>",
        "ip-match": {
          "ip-protocol": <integer>
        },
        "tcp-destination-port": <integer>,
        "udp-destination-port": <integer>
      },
      "instructions": {
        "instruction": [
          {
            "order": 0,
            "apply-actions": {
              "action": [
                {
                  "order": 0,
                  "set-queue-action": {
                    "queue-id": <queue_id>
                  }
                },
                {
                  "order": 1,
                  "output-action": {
                    "output-node-connector": "<port_number>"
                  }
                }
              ]
            }
          }
        ]
      }
    }
  ]
}

2. **Blocking or Dropping Rule:**  

{
  "flow-node-inventory:flow": [
    {
      "id": "<unique_id>",
      "priority": <integer>,
      "table_id": <integer>,
      "flow-name": "<descriptive_name>",
      "hard-timeout": <integer>,
      "idle-timeout": <integer>,
      "match": {
        "ethernet-match": {
          "ethernet-type": {
            "type": <integer>
          }
        },
        "ipv4-source": "<ip_address/mask>",
        "ipv4-destination": "<ip_address/mask>"
      },
      "instructions": {
        "instruction": [
          {
            "order": 0,
            "apply-actions": {
              "action": [
                {
                  "order": 0,
                  "drop-action": {}
                }
              ]
            }
          }
        ]
      }
    }
  ]
}

3. **Port-Based Forwarding Rule:**  

{
  "flow-node-inventory:flow": [
    {
      "id": "<unique_id>",
      "priority": <integer>,
      "table_id": <integer>,
      "flow-name": "<descriptive_name>",
      "hard-timeout": <integer>,
      "idle-timeout": <integer>,
      "match": {
        "in-port": <port_number>
      },
      "instructions": {
        "instruction": [
          {
            "order": 0,
            "apply-actions": {
              "action": [
                {
                  "order": 0,
                  "output-action": {
                    "output-node-connector": "<port_number>"
                  }
                }
              ]
            }
          }
        ]
      }
    }
  ]
}

Field Descriptions: 
id: A number representing a unique identifier for the flow (0 for default).
priority: Priority level (higher numbers indicate higher priority).
table_id: An integer representing the flow table identifier (0 for default).
flow-name: A short, descriptive flow name.
hard-timeout: Timeout in seconds after which the flow is removed (0 for default).
idle-timeout: Timeout in seconds after which the flow is removed if there's no activity (0 for default).
ethernet-type: Ethernet protocol type (e.g., 2048 for IPv4).
ipv4-destination: IPv4 address in CIDR notation (e.g., 10.0.0.1/32 (optional)).
ipv4-source: Source IP address (optional).
ip-protocol: Use "ip-match" and "ip-protocol" when specifying specific transport layer protocols (e.g., 6 for TCP, 17 for UDP, 1 for ICMP (optional)).
tcp-source-port: TCP port number (optional).
tcp-destination-port: TCP port number (optional).
udp-source-port: UDP port number (optional).
udp-destination-port: UDP port number (optional).
in-port: A value representing incoming interface port number (optional).
output-action: Use "output-action" and "output-node-connector" to specify the output port number (optional).
set-queue-action: Use "set-queue-action" and "queue-id" when the intent specifies assigning traffic to a queue (The "queue-id" is an integer (0 for default)).
drop-action: Use {} to indicate packet dropping.

RULES: 
Each id must be unique.
Set priority values appropriately (higher for critical rules, lower for defaults).
Use valid match conditions (ipv4-destination, tcp-destination-port, ipv4-source, in-port) depending on the intent type.
Ensure valid ODL-compliant JSON syntax.
Avoid duplicate keys and empty fields.
Verify JSON structure for correctness before responding.
Always respond with valid JSON only, with no additional text, comments, or explanations.
If the intent cannot be mapped, return an empty JSON object {}."""

In [96]:
my_models = [
"codestral",
"huihui_ai/qwq-abliterated",
"huihui_ai/qwq-fusion",
"qwq"
]

default_model = "llama2"

In [97]:
continuous_learning = False
num_examples = [0, 1, 3, 6, 9]
use_case = "formal_spec"
ollama_embedding_url = "http://localhost:11434"
ollama_server_url = "http://localhost:11435"  

In [98]:
ollama_emb = OllamaEmbeddings(
    model=default_model,
    base_url=ollama_embedding_url,
)

client = Client(host=ollama_server_url , timeout=120)

# Load custom dataset from CSV
custom_dataset = pd.read_csv('custom_ODL_intent_dataset_formal_Final.csv')

# Ensure proper column names and format
if not {'instruction', 'output'}.issubset(custom_dataset.columns):
    raise ValueError("The dataset must have 'instruction' and 'output' columns.")

# Split into train and test (50/50 split for example)
trainset, testset = train_test_split(custom_dataset, test_size=0.5, random_state=42, shuffle=True)

# SYSTEM PROMPT (Manually define)
SYSTEM_PROMPT = NEW_PROMPT3

In [99]:
for num_examples in num_examples:
    for model in my_models:
        # too much examples in big model cause timeout
        if num_examples > 1 and model in ["qwen2:72b", "llama3.1:70b"]:
            continue
        # this test has same result when add example = false
        if continuous_learning and num_examples == 0:
            continue

        # create example selector with one example, then clear the data and add all examples
        # this is a trick to reset data and remove data from continuous learning in previous run
        example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
            [{"instruction": trainset.iloc[0]["instruction"], "output": trainset.iloc[0]["output"]}],
            ollama_emb,
            Chroma,
            input_keys=["instruction"],
            k=num_examples,
            vectorstore_kwargs={"fetch_k": min(num_examples, len(trainset))}
            )

        # Clear and add all remaining examples from the trainset
        example_selector.vectorstore.reset_collection()
        for _, row in trainset.iterrows():
            example_selector.add_example({
                "instruction": row["instruction"],
                "output": row["output"]
            })
        

        print("\n\n=====================================\n")
        print(f"Start eval on use case: {use_case}, model: {model}, num context examples: {num_examples}, continuous learning: {continuous_learning}")
        print("\n")
        #sys.stdout.flush()

        correct = 0
        total = 0
        processing_times = []
        # for testcase in testset:
        #     intent = testcase["instruction"]
        #     expected_output = testcase["output"]
        for _, testcase in testset.iterrows():
            intent = testcase["instruction"]
            expected_output = testcase["output"]
        
            system_prompt = SYSTEM_PROMPT

            while True:
                try:
                    time.sleep(0.1)
                    current_time = time.time()
                    if num_examples > 0:
                        # we do not use FewShotPromptTemplate due to JSON serialization issues
                        # https://github.com/langchain-ai/langchain/issues/4367
                        examples = example_selector.select_examples({"instruction": intent})
                        example_str = "\n\n\n".join(map(lambda x: "Input: " + x["instruction"] + "\n\nOutput: " + x["output"], examples))
                        system_prompt += example_str + "\n\n\n"  
                    
                    response = client.generate(model=model,
                        options={
                            'temperature': 0.6,
                            'num_ctx': 8192,
                            'top_p': 0.3,
                            'num_predict': 1024,
                            'num_gpu': 99,
                            },
                        stream=False,
                        system=system_prompt,
                        prompt=intent,
                        format='json'
                    )
                    actual_output = response['response']

                    proc_time_s = (time.time() - current_time)
                    processing_times.append(proc_time_s)
                    break
                except Exception as e:
                    print("Exception on Input: ", e)
                    sys.stdout.flush()
                    continue

            try:
                expected_output = json.loads(expected_output)
                actual_output = json.loads(actual_output)



                #num_correct_translation, total_translation = compare_result(expected_output, actual_output)

                #if continuous_learning and num_examples > 0 and num_correct_translation < total_translation:
                    #example_selector.add_example({"instruction": intent, "output": json.dumps(expected_output)})
                    #print(f"Wrong output, add example!")

                #if num_correct_translation == 0:
                    #print(f"Input: {intent}")
                    #print(f"Expected: {expected_output}")
                    #print(f"Actual: {actual_output}")
                
                print(f"Input: {intent}")
                print("\n\nExpected Output\n")
                print(f"Expected: {expected_output}")
                print("\n\nActual Output\n")
                print(f"Actual: {actual_output}")
                

                print("\n\nResult Start-JSON diff method\n\n")
                print(f"Diff: {jsondiff.diff(expected_output, actual_output)}")
                print("\n\nResult End-JSON diff method\n")
                #correct += num_correct_translation
                #total += total_translation

                print("=====================================")
                #print(f"Corrects: {correct}, total: {total}, percent: {(correct/total)*100}, proc time: {proc_time_s}")
                #sys.stdout.flush()
            except Exception as e:
                print("Exception found: ", e)

        print("\n========End of the testcase===========")
        print("\n========###################===========")
        print("\n========*******************===========\n\n\n")
        #print(f"Finish eval on use case: {use_case}, model: {model}, num context examples: {num_examples}, continuous learning: {continuous_learning}, testcases: {total}, accuracy: {round((correct/total)*100, 2)}  avg proc time: {round(np.average(processing_times), 1)}, std: \t{round(np.std(processing_times), 1)}")
        #print(f"Finish eval on use case: {use_case}, model: {model}, num context examples: {num_examples}, continuous learning: {continuous_learning}, testcases: {total}, accuracy: {round((correct/total)*100, 3)}  avg proc time: {round(np.average(processing_times), 1)}, std: \t{round(np.std(processing_times), 1)}, processing time array: {[round(x, 2) for x in processing_times]}")           
        
        sys.stdout.flush()




Start eval on use case: formal_spec, model: codestral, num context examples: 0, continuous learning: False


Input: Route UDP traffic (port 80) targeting 10.0.0.3/32 through port 2 with traffic assigned to queue 1.


Expected Output

Expected: {'flow-node-inventory:flow': [{'id': '1', 'priority': 198, 'table_id': 0, 'flow-name': 'udp_port80_queue1_to_h3', 'instructions': {'instruction': [{'order': 0, 'apply-actions': {'action': [{'order': 0, 'set-queue-action': {'queue-id': 1}}, {'order': 1, 'output-action': {'output-node-connector': '2'}}]}}]}, 'match': {'ethernet-match': {'ethernet-type': {'type': 2048}}, 'ip-match': {'ip-protocol': 17}, 'ipv4-destination': '10.0.0.3/32', 'udp-destination-port': 80}}]}


Actual Output

Actual: {'flow-node-inventory:flow': [{'id': '1', 'priority': 50, 'table_id': 0, 'flow-name': 'UDP_Traffic_Rule', 'hard-timeout': 0, 'idle-timeout': 0, 'match': {'ethernet-match': {'ethernet-type': {'type': 2048}}, 'ipv4-destination': '10.0.0.3/32', 'ip-match': {'ip

Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17





Start eval on use case: formal_spec, model: codestral, num context examples: 1, continuous learning: False




Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17


Input: Route UDP traffic (port 80) targeting 10.0.0.3/32 through port 2 with traffic assigned to queue 1.


Expected Output

Expected: {'flow-node-inventory:flow': [{'id': '1', 'priority': 198, 'table_id': 0, 'flow-name': 'udp_port80_queue1_to_h3', 'instructions': {'instruction': [{'order': 0, 'apply-actions': {'action': [{'order': 0, 'set-queue-action': {'queue-id': 1}}, {'order': 1, 'output-action': {'output-node-connector': '2'}}]}}]}, 'match': {'ethernet-match': {'ethernet-type': {'type': 2048}}, 'ip-match': {'ip-protocol': 17}, 'ipv4-destination': '10.0.0.3/32', 'udp-destination-port': 80}}]}


Actual Output

Actual: {'flow-node-inventory:flow': [{'id': '2', 'priority': 200, 'table_id': 0, 'flow-name': 'udp_port80_queue1_to_h3', 'instructions': {'instruction': [{'order': 0, 'apply-actions': {'action': [{'order': 0, 'set-queue-action': {'queue-id': 1}}, {'order': 1, 'output-action': {'output-node-connector': '2'}}]}}]}, 'match': {'ethernet-match': {'ethernet-type': {'type': 2048}},

...

Input: Ensure TCP packets on port 80 addressed to 10.0.0.3 are sent through port 2.


Expected Output

Expected: {'flow-node-inventory:flow': [{'id': '1', 'priority': 110, 'table_id': 0, 'flow-name': 'high_bandwidth_path_to_h3', 'hard-timeout': 0, 'idle-timeout': 0, 'match': {'ethernet-match': {'ethernet-type': {'type': 2048}}, 'ipv4-destination': '10.0.0.3/32', 'ip-match': {'ip-protocol': 6}, 'tcp-destination-port': 80}, 'instructions': {'instruction': [{'order': 0, 'apply-actions': {'action': [{'order': 0, 'output-action': {'output-node-connector': '2'}}]}}]}}]}


Actual Output

Actual: {'flow-node-inventory:flow': [{'id': '1', 'priority': 110, 'table_id': 0, 'flow-name': 'forward_tcp_port80_to_h3', 'hard-timeout': 0, 'idle-timeout': 0, 'match': {'ethernet-match': {'ethernet-type': {'type': 2048}}, 'ipv4-destination': '10.0.0.3/32', 'ip-match': {'ip-protocol': 6}, 'tcp-destination-port': 80}, 'instructions': {'instruction': [{'order': 0, 'apply-actions': {'action': [{'order': 0, 'out

Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17


Input: Forward all traffic not matching higher-priority rules through port 1.


Expected Output

Expected: {'flow-node-inventory:flow': [{'id': '1', 'priority': 80, 'table_id': 0, 'hard-timeout': 0, 'match': {'ethernet-match': {'ethernet-type': {'type': 2048}}}, 'flow-name': 'default_traffic', 'instructions': {'instruction': [{'order': 0, 'apply-actions': {'action': [{'order': 0, 'output-action': {'output-node-connector': '1'}}]}}]}, 'idle-timeout': 0}]}


Actual Output

Actual: {'flow-node-inventory:flow': [{'id': '1', 'priority': 1, 'table_id': 0, 'flow-name': 'default_flow', 'hard-timeout': 0, 'idle-timeout': 0, 'match': {}, 'instructions': {'instruction': [{'order': 0, 'apply-actions': {'action': [{'order': 0, 'output-action': {'output-node-connector': '1'}}]}}]}}]}


Result Start-JSON diff method


Diff: {'flow-node-inventory:flow': {0: {'priority': 1, 'match': {replace: {}}, 'flow-name': 'default_flow'}}}


Result End-JSON diff method



Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17


Input: Route default traffic via interface 1.


Expected Output

Expected: {'flow-node-inventory:flow': [{'id': '1', 'priority': 80, 'table_id': 0, 'hard-timeout': 0, 'match': {'ethernet-match': {'ethernet-type': {'type': 2048}}}, 'flow-name': 'default_traffic', 'instructions': {'instruction': [{'order': 0, 'apply-actions': {'action': [{'order': 0, 'output-action': {'output-node-connector': '1'}}]}}]}, 'idle-timeout': 0}]}


Actual Output

Actual: {'flow-node-inventory:flow': [{'id': '1', 'priority': 80, 'table_id': 0, 'flow-name': 'default_traffic', 'hard-timeout': 0, 'idle-timeout': 0, 'match': {'ethernet-match': {'ethernet-type': {'type': 2048}}}, 'instructions': {'instruction': [{'order': 0, 'apply-actions': {'action': [{'order': 0, 'output-action': {'output-node-connector': '1'}}]}}]}}]}


Result Start-JSON diff method


Diff: {}


Result End-JSON diff method

Input: Route all packets received on port 1 to interface 2.


Expected Output

Expected: {'flow-node-inventory:flow': [{'i